We will perform the analysis on slow running queries on the Yugabyte Managed cluster.

In [109]:
# Place your token in a file and upload the file on Google drive. This will help to avoid sharing your own key while sharing code

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/

fname="kapil-test-single-region-1_api_token.txt"

f= open(fname, 'r') 
api_token_code=str.strip((f.read()))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks


In [110]:
import requests
import json
import pandas as pd

account_id = "69e28ad7-842c-4001-8cfa-c5c20610f108"

url = "https://cloud.yugabyte.com/api/public/v1/accounts/"+account_id+"/projects"

headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer "+ api_token_code
    }

response = requests.request("GET", url, headers=headers)

In [111]:
project_info = json.loads(response.text)
project_id=project_info['data'][0]['id']
#print ("Project ID: " + project_id)

In [112]:
url = "https://cloud.yugabyte.com/api/public/v1/accounts/"+account_id+"/projects/"+project_id+"/clusters"

response = requests.request("GET", url, headers=headers)

In [113]:
cluster_info = json.loads(response.text)
cluster_id=cluster_info['data'][0]['info']['id']
#cluster_id

In [114]:
url = "https://cloud.yugabyte.com/api/public/v1/accounts/"+account_id+"/projects/"+project_id+"/clusters/"+cluster_id+"/slow-queries"

response = requests.request("GET", url, headers=headers)

In [115]:
slow_queries = json.loads(response.text)
all_queries=slow_queries['data']['ysql']['queries']
df=pd.DataFrame(all_queries)

In [116]:
print("Total number of slow queries : " + str(df.shape[0])) 

Total number of slow queries : 37


In [117]:
print ("Long running queries")
df.sort_values(by='max_time', ascending=False).head()

Long running queries


,queryid,query,rolname,datname,calls,local_blks_hit,local_blks_written,max_time,mean_time,min_time,rows,stddev_time,total_time
29,1499088806970351965,insert into pg_class_1 select * from pg_class_1,admin,northwind,20,0,0,1.781158e+06,161206.960732,102.344461,3369306,408312.884649,3.224139e+06
18,-6880009419006274021,CREATE INDEX idx_customer_name ON customer ((c...,admin,yugabyte,3,0,0,8.565282e+03,6876.753465,4630.478473,0,1654.081909,2.063026e+04
9,788848221759902645,create database northwind,admin,yugabyte,3,0,0,6.715629e+03,6636.116797,6559.313454,0,63.844255,1.990835e+04
28,-694261495543074897,CREATE TABLE pg_class_1 (\n relname name NO...,admin,northwind,2,0,0,4.749896e+03,4749.896407,4749.896407,0,0.000000,9.499793e+03
27,8353115169181364800,"CREATE TABLE OORDER ( \no_w_id int NOT NULL,\n...",admin,yugabyte,3,0,0,4.627427e+03,4539.889112,4442.884462,0,75.636008,1.361967e+04


In [118]:
print ("Most frequent running queries")
df.sort_values(by='calls', ascending=False).head()

Most frequent running queries


,queryid,query,rolname,datname,calls,local_blks_hit,local_blks_written,max_time,mean_time,min_time,rows,stddev_time,total_time
31,-2471462977972719462,"INSERT INTO STOCK VALUES ($1, $2, $3, $4, $5, ...",admin,yugabyte,42356,0,0,650.506184,66.226741,26.348156,5421568,44.452293,2.805100e+06
3,-790177869016502498,"INSERT INTO ORDER_LINE VALUES ($1, $2, $3, $4,...",admin,yugabyte,38899,0,0,1109.992853,19.118910,7.511300,2489536,16.147798,7.437065e+05
35,-2826215450893820657,"INSERT INTO ORDER_LINE VALUES ($1, $2, $3, $4,...",admin,yugabyte,37271,0,0,132.236771,13.300191,4.360779,1192672,12.481882,4.957114e+05
20,-7029554077961096057,"INSERT INTO OORDER VALUES ($1, $2, $3, $4, $5,...",admin,yugabyte,34840,0,0,1594.672490,11.907684,2.257237,278720,26.755608,4.148637e+05
33,-4335637899996284123,"INSERT INTO ORDER_LINE VALUES ($1, $2, $3, $4,...",admin,yugabyte,25702,0,0,90.624508,10.066652,2.827980,411232,11.233191,2.587331e+05


In [119]:
print ("Local Buffer Usage")
df.sort_values(by='local_blks_hit', ascending=False).head()

Local Buffer Usage


,queryid,query,rolname,datname,calls,local_blks_hit,local_blks_written,max_time,mean_time,min_time,rows,stddev_time,total_time
0,-8946192826943326133,"CREATE TABLE STOCK ( \ns_w_id int NOT NULL,\ns...",admin,yugabyte,3,0,0,4363.914851,1577.320351,177.448873,0,1970.427179,4731.961053
19,-2941792897847509906,"CREATE TABLE ITEM ( \ni_id int NOT NULL,\ni_na...",admin,yugabyte,3,0,0,4282.170139,1569.323923,199.937708,0,1918.301156,4707.971768
21,-6220839165755395970,"INSERT INTO NEW_ORDER VALUES ($1, $2, $3),($4,...",admin,yugabyte,2124,0,0,70.928684,8.017631,1.331900,8496,11.998513,17029.448796
22,-2529188128542357506,CREATE UNIQUE INDEX idx_order ON oorder ((o_w_...,admin,yugabyte,3,0,0,3468.624595,3418.435073,3380.395233,0,37.029863,10255.305219
23,-8987939333074667197,"INSERT INTO ORDER_LINE VALUES ($1, $2, $3, $4,...",admin,yugabyte,4605,0,0,228.746078,33.095230,13.783190,589440,22.107616,152403.535406
